In [1]:
import os

In [2]:
%pwd

'c:\\Users\\umair\\Desktop\\ML\\waterQualityPrediction\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from waterQualityPrediction.constants import *
from waterQualityPrediction.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
          
    def get_data_transformation_config(self) -> DataTransformationConfig:   
        config = self.config.data_preprocess 

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config


In [8]:
import scipy
import pandas as pd
from waterQualityPrediction import logger
from sklearn.model_selection import train_test_split

C:\Users\umair\AppData\Local\Temp\ipykernel_12548\3814393511.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    def preprocess_data(self):
        data = pd.read_csv(self.config.data_path)
        logger.info("Imputing missing values in the dataset")
        data['ph'] = data.groupby(['Potability'])['ph'].transform('mean')
        data['Sulfate'] = data.groupby(['Potability'])['Sulfate'].transform('mean')
        data['Trihalomethanes'] = data.groupby(['Potability'])['Trihalomethanes'].transform('mean')
        # Removing rows where "ph" is less than or equal to 0
        logger.info("Removing row contain 0")
        data = data[data["ph"] > 0]
        
        # Applying Box-Cox transformation to all columns
        logger.info("Normalizing the data with Box-Cox")
        data["ph"], fitted_lambda = scipy.stats.boxcox(data["ph"])
        data["Hardness"], fitted_lambda = scipy.stats.boxcox(data["Hardness"])
        data["Solids"], fitted_lambda = scipy.stats.boxcox(data["Solids"])
        data["Chloramines"], fitted_lambda = scipy.stats.boxcox(data["Chloramines"])
        data["Sulfate"], fitted_lambda = scipy.stats.boxcox(data["Sulfate"])
        data["Conductivity"], fitted_lambda = scipy.stats.boxcox(data["Conductivity"])
        data["Organic_carbon"], fitted_lambda = scipy.stats.boxcox(data["Organic_carbon"])
        data["Trihalomethanes"], fitted_lambda = scipy.stats.boxcox(data["Trihalomethanes"])
        data["Turbidity"], fitted_lambda = scipy.stats.boxcox(data["Turbidity"])
        
        #saving data after preprocessing
        data.to_csv(os.path.join(self.config.root_dir, "water-quality-processed.csv"), index=False)


    def train_test_spliting(self):
        # data = pd.read_csv(self.config.root_dir)
        data = pd.read_csv(os.path.join(self.config.root_dir, "water-quality-processed.csv"))

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.preprocess_data()
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-01-25 21:25:13,942: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-25 21:25:13,945: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-25 21:25:13,947: INFO: common: created directory at: artifacts]
[2024-01-25 21:25:13,950: INFO: common: created directory at: artifacts/preprocess]
[2024-01-25 21:25:13,968: INFO: 901800738: Imputing missing values in the dataset]
[2024-01-25 21:25:13,980: INFO: 901800738: Removing row contain 0]
[2024-01-25 21:25:13,984: INFO: 901800738: Normalizing the data with Box-Cox]


c:\Users\umair\miniconda3\envs\water\lib\site-packages\scipy\stats\_morestats.py:1121: UserWarning: The optimal lambda is 829.5925732760456, but the returned lambda is theconstrained optimum to ensure that the maximum or the minimum of the transformed data does not cause overflow in float64.
  lmax = boxcox_normmax(x, method='mle', optimizer=optimizer)
c:\Users\umair\miniconda3\envs\water\lib\site-packages\scipy\stats\_morestats.py:1121: UserWarning: The optimal lambda is 226.9189068301042, but the returned lambda is theconstrained optimum to ensure that the maximum or the minimum of the transformed data does not cause overflow in float64.
  lmax = boxcox_normmax(x, method='mle', optimizer=optimizer)


[2024-01-25 21:25:14,546: INFO: 901800738: Splited data into training and test sets]
[2024-01-25 21:25:14,548: INFO: 901800738: (2457, 10)]
[2024-01-25 21:25:14,551: INFO: 901800738: (819, 10)]
(2457, 10)
(819, 10)
